## Automatizované hľadanie parametrov modelu 

V minulom cvičení sme si ukázali ako funguje ladenie modelu pomocou nastavovania hodnôt jeho parametrov. Tento proces môže prebiehať aj automatizovane - generovaním množstva modelov s rôznymi parametrami a ich vyhodnocovaním. Cieľom tejto úlohy je demonštrovať ako takýmto spôsobom hľadať najvhodnejšie paramertre klasifikačného modelu. V nasledujúcej úlohe demonštrujeme, ako takýmto spôsobom hľadať optimálne parametre pre model k-NN.

Rovnako ako v predchádzajúcej úlohe budeme pracovať s datasetom Titanic, ktorý sme predspracovali na cvičení č. 7. Pre účely ladenia parametrov ho predspracujeme rovnakým spôsobom (použitím rovnakých transformácií) ako v predošlom cvičení.

Najprv teda naimportujeme všetky potrebné knižnice.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

Načítame teda do dátového rámca `titanic` predspracované dáta z datasetu Titanic, z cvičenia č. 7. Nachádzajú sa v súbore `../data/titanic-processed.csv`.

In [ ]:
titanic = pd.read_csv("../data/titanic-processed.csv")
titanic.head()

Keďže budeme vytvárať rovnaký model (k-NN)ako v predošlých úlohách, niektoré z atribútov nepoužijeme (tie, ktoré obsahujú priveľa chýbajúcich hodnôt, resp. tie, ktoré obsahujú príliš veľké množstvo kategorických hodnôt), iné transformujeme pomocou One Hot Endoceru, alebo priradením číselných indexov.

In [ ]:
titanic = titanic.drop(columns=['cabin','deck','ticket','title'])
titanic['sex'] = titanic['sex'].map({"male": 0, "female": 1})
titanic['has_family'] = titanic['has_family'].map({False: 0, True: 1})
titanic['fare_ordinal'] = titanic['fare_ordinal'].map({"normal": 0, "more expensive": 1, "most expensive": 2})
titanic['age_ordinal'] = titanic['age_ordinal'].map({"child": 0, "young": 1, "adult": 2, "old": 3}) 
titanic = pd.get_dummies(titanic, columns=['embarked', 'title_short'])

In [ ]:
titanic.head()

Keďže vytváram model k-NN, je vhodné dáta predspracovať aj normalizáciou. Použijeme teda znova `MinMaxScaler`, aby sme atribúty naškálovali na jednotnú mierku. 

In [ ]:
from sklearn.preprocessing import MinMaxScaler 

scaler = MinMaxScaler() 
titanic = pd.DataFrame(scaler.fit_transform(titanic), index=titanic.index, columns=titanic.columns)
titanic.head() 

Na takto predspracovanej množine už môžeme vyskúšať natrénovať klasifikačný model. Podobne ako v predchádzajúcom cvičení najprv rozdelíme dáta do matice príznakov a vektora hodnôt cieľového atribútu.
Cieľovým atribútom v tejto úlohe je `survived` (vyjadruje, či daný pasažier nehodu prežil alebo nie). Cieľový atribút teda bude tvoriť vektor hodnôt `y` a zostávajúce stĺpce maticu príznakov `X`.

In [ ]:
X_titanic = titanic.drop('survived', axis=1) # vytvoríme maticu príznakov - použijeme všetky stĺpce okrem cieľového atribútu a uložíme do X_titanic
y_titanic = titanic['survived'] # vytvoríme vektor hodnôt cieľového atribútu ako stĺpec 'survived'

print(X_titanic.shape) # pre kontrolu môžeme vypísať rozmery matice hodnôt a vektora cieľového atribútu
print(y_titanic.shape)

Teraz rozdelíme dáta do trénovacej a testovacej množiny. Na rozdelenie dát na trénovacie a testovacie použijeme funkciu `train_test_split()`, tesovacia množina bude v pomere 30/70 k trénovacej.

In [ ]:
from sklearn.model_selection import train_test_split # importujeme funkciu train_test_split()
X_train, X_test, y_train, y_test = train_test_split(X_titanic, y_titanic, test_size=0.3, random_state=1) # rozdelíme dataset do trénovacej a testovacej časti, tak že testovacia bude 30% z celkového datasetu

Vytvoríme objekt k-NN modelu. Bez špecifických parametrov - tie tentoraz budeme hľadať pomocou funkcie GridSearch. 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier # importujeme knižnicu pre K-NN

knn = KNeighborsClassifier() # inicializujeme klasifikátor

### GridSearch pre hľadanie optimálnych nastavení algoritmov

Pomocou funkcie `GridSearchCV` môžeme automatizovať hľadanie optimálnych parametrov algoritmov. Grid Search je prístup, ktorý automaticky vytvorí množinu modelov s rôznymi nastaveniami, ktoré validuje použitím krížovej validácie. 

#### Nastavenie parametrov Grid Search

Funkcii Grid Search v Scikit-learn špecifikujeme niekoľko vstupných parametrov, ktoré potom definujú, akým spôsobom sa automatizované testovanie parametrov modelu uskutoční. 

V príklade nižšie vyskúšame nájsť pomocou Grid Search optimálnu hodnotu parmetra `k` pre model k-NN. Najprv definujeme rozsah hodnôt parametra `k`, ktoré chceme testovať. 

In [ ]:
from sklearn.model_selection import GridSearchCV # importujeme potrebné knižnice

# definujeme hodnoty parametrov, ktoré sa budú prehľadávať
# pre parameter k vygenerujeme rozsah 1 až 50

k_range = list(range(1, 50))
print(k_range)

Vytvoríme pole parametrov modelu. Tu si musíme dať pozor - pole pre jednotlivé parametre musíme vytvoriť tak, aby názvy parametrov zodpovedali názvom parametrov jednotlivých modelov. 

V tomto príklade sme vygenerovali pole celých čísel, ktoré chceme použiť ako rôzne hodnoty pre testovanie parametra `k`. V k-NN modeli sa tento parameter nazýva `n_neighbors` (pri nastavovaní parametra klasifikátora k-NN sme modely vytvárali ako napr. `KNeighborsClassifier(n_neighbors = 3)`), preto namapujeme do premennej, ktorá bude uchovávať kolekciu parametrov(`param_grids`) pole týchto hodnôt, ktoré priradíme parametru `n_neighbors`.

In [ ]:
# vytvoríme tzv. parameter grid: namapujeme vygenerované hodnoty do poľa parametrov
# v tomto prípade vytvoríme parameter n_neighbors, ktorému priradíme pole jeho skúmaných hodnôt

param_grid = dict(n_neighbors=k_range)
print(param_grid)

Teraz, keď máme nastavené pole parametrov, ktoré chceme preskúmať, spustíme Grid Search. `GridSearchCV` má nasledovné parametre:
* `estimator` - model, ktorý chceme trénovať (v našom prípade `knn`)
* `param_grid` - kolekcia parametrov modelu a zoznamov ich hodnôt - pozor, pole parametrov musí byť kompatibilné s parametrami modelu!
* `cv` - faktor kížovej validácie
* `scoring` - metrika používaná na vyhodnotenie v krížovej validácii modelov (napr. `accuracy`, `precision`, `recall` atď.)

In [ ]:
# aplikujeme Grid Search - nastavíme parametre:
# model - knn
# pole parametrov - param_grid
# budeme používať 5-násobnú krížovú validáciu 
# na vyhodnotenie použijeme metriku accuracy

grid = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='accuracy') # nastavíme parametre Grid Searchu
grid.fit(X_train, y_train) # aplikujeme Grid Search na trénovacích dátach

#### Vyhodnotenie výsledkov Grid Search

Teraz máme natrénovanú množinu klasifikátorov s rôznymi nastaveniami a cez rôzne funkcie objektu `grid` sa môžeme pozrieť na konrétne výsledky modelov s rôznymi hodnotami vstupného parametra `k`.

Pomocou `best_params_` sa môžeme pozrieť, ktorý model dosiahol najlepšie výsledky. 

In [ ]:
print("Najlepšie parametre sú:")
print()
print(grid.best_params_)
print()
print(grid.best_score_)

Pomocou `cv_results_` vieme získať rôzne metriky:

* `mean_test_score` - priemerné skóre (definované ako parameter Grid Search-u)  
* `std_test_score` - štandardná odchýlka skóre     
* `rank_test_score` - poradie v skóre na testovacích dátach    
* `mean_train_score` - priemerné skóre (na trénovacích podmnožinách)   
* `std_train_score` - štandardná odchýkla skóre na trénovacích dátach   
* `mean_fit_time` - priemerný čas trénovania modelu     
* `std_fit_time` - štandardná odchýlka času trénovania modelu       
* `mean_score_time` - priemerný čas vyhodnocovania neznámych príkladov   
* `std_score_time` - štandardná odchýlka času vyhodnocovania    
* `params` - parametre modelov

Môžeme sa teda pozrieť, aké rôzne metriky a informácie objekt `cv_results_` uchováva:

In [ ]:
sorted(grid.cv_results_.keys())

Môžeme sa teda pozrieť na konkrétne výsledky konkrétnych modelov.

In [ ]:
print(grid.cv_results_["mean_test_score"][24]) # výsledky pre konkrétnu metriku a pre konkrétny model

Môžeme sa samozrejme pozrieť na to, aké výsledky dosiahli všetky modely naraz. Vypíšeme priemerné skóre krížovej validácie, jeho štandardnú odchýlku a parametre daného modelu. Aby bol výpis lepšie čitateľný, je potrebné výpis rozumne naformátovať. 

In [ ]:
# môžeme sa pozrieť na kompletné výsledky

print("Jednotlivé skóre pre jednotlivé hodnoty parametra k:")
print()
means = grid.cv_results_['mean_test_score'] # do premennej means priradíme výsledky priemerov testovacieho skóre
stds = grid.cv_results_['std_test_score'] # do premennej stds priradíme zoznam štandardných odchýliek
params = grid.cv_results_['params']

for mean, std, params in zip(means, stds, params): # pre všetky záznamy vypíšeme naformátovaný výstup - zip mapuje rovnaké indexy viacerých kontajnerov/polí aby mohli byť používané ako jedna entita
    print("%0.3f (+/-%0.03f) pre hodnotu %s" % (mean, std, params)) # naformátujeme výpis
print()

Môžeme sa (pomocou prístupu k jednotlivým prvkom výsledkov) pozrieť aj na konkrétne výsledky zvoleného modelu. Môžeme aj preskúmať jednotlivé čiastkové výsledky z krížovej validácie daného modelu. Po špecifikácii parametra teda môžeme špecifikovať aj index, pomocou ktorého pristúpime ku konkrétnemu modelu.

In [ ]:
# môžeme preskúmať jednotlivé modely a ich konkrétne výsledky

print('Parameter k pre model 0:')
print(grid.cv_results_["params"][0])

# skóre modelu s indexom 0 (k=1) pre jednotlivé splity krížovej validácie
print()
print('CV skóre pre model 0:')
print(grid.cv_results_["split0_test_score"][0])
print(grid.cv_results_["split1_test_score"][0])
print(grid.cv_results_["split2_test_score"][0])
print(grid.cv_results_["split3_test_score"][0])
print(grid.cv_results_["split4_test_score"][0])

# Priemerné skóre modelu s indexom 0
print()
print('Priemerné skóre pre model 0')
print(grid.cv_results_["mean_test_score"][0])

#### Vizualizácia závislosti hodnoty parametra k na skóre

Pre lepšie pochodpenie závislosti jedného parametra od výsledného skóre modelu môžeme vizualizovať. Vykreslenie závislosti presnosti a hodnoty parametra `k` a metriky Accuracy na testovacej množine potom v tomto prípade veľmi jednoducho vizualizujeme pomocou matplotlib. 

In [ ]:
# pomocou matplotlib vykreslíme závislosť hondôt parametra k a skóre teda medzi týmito dvoma veličinami:

# YOUR CODE HERE
plt.plot( # YOUR CODE HERE )
plt.xlabel(' ... ')
plt.ylabel(' ... ')

### Simultánne hľadanie viacerých parametrov

Metóde Grid Search vieme špecifikovať viacero parametrov súčasne. Algoritmus tak bude hľadať všetky kombinácie definovaných parametrov. 

Skúsime hľadať kombináciu aj ostatných parametrov. Pre algoritmus k-NN vieme nastavovať ešte parametre špecifikujúce váhovanie vzdialenosti alebo použitú metriku. Vytvoríme teda ďalšie zoznam hodnôt parametra `weights` a zoznam, ktorý zodpovedá hodnotám parametra `metric`.

In [ ]:
# vytvorte zoznamy parametrov pre váhy a metriky algoritmu k-NN

weights_range = #  
metric_range = #  

Oba z týchto zoznamov potom pridáme spolu so zoznamom `k_range` do zoznamu parametrov. 

Parameter algoritmu kNN, ktorý špecifikuje váhovanie sa nazýva `weights` a parameter definujúci metriky sa nazýva `metric`. Priradíme im zoznamy ich hodnôt, ktoré chceme preskúmať. Do kolekcie `param_grid` teda vložíme polia hodnôt pre jednotlivé parametre.

In [ ]:
param_grid = dict(n_neighbors=k_range, weights=weights_range, metric=metric_range)
print(param_grid)

Teraz rovnakým spôsobom spustíme Grid Search - špecifikujeme model, zoznam parametrov, parameter nastavenia krížovej validácie a definujeme metriku vyhodnocovania. 

In [ ]:
grid = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='accuracy') # nastavíme parametre Grid Searchu
grid.fit(X_train, y_train) # naučíme Grid Search na trénovacích dátach

Rovnakým spôsobom môžeme opäť pristúpiť k výsledkom. 

Pozrime sa na najlepší z modelov a jeho výsledok a potom rovnakým spôsobom ako v predošlej úlohe vypíšeme kompletné výsledky.

In [ ]:
print("Najlepšia kombinácia parametrov je:")
print(grid.best_params_)
print()
print("A hodnota presnosti modelu pri tejto kombinacii je:")
print(grid.best_score_)

In [ ]:
print("Kompletné výsledky:")
print()
means = grid.cv_results_['mean_test_score']
for mean, params in zip(means, grid.cv_results_['params']):
    print("%0.3f pre hodnotu %r" % (mean, params))
print()

Použitím `GridSearchCV` sme natrénovali modely s rôznymi parametrami na trénovacej množine. Zároveň, použitím krížovej validácie sme ich na trénovacej množine aj validovali. Identifikovali sme tak najlepšie parametre modelu. Ak chceme model otestovať na testovacej množine, aby sme tak overili jeho kvalitu alebo aby sme model mohli použiť pre predikciu nových, neoznačených príkladov, musíme model s identifikovanými parametrami opäť natrénovať. Potom ho môžeme otestovať na testovacej množine a vypísať kontigenčnú tabuľku klasifikácie. 

### Úloha 12.1

Natrénujte model s najlepšími parametrami na trénovacej množine a otestujte na testovacej. Vypíšte kontigenčnú tabuľku výsledkov (confusion matrix). 

In [ ]:
# YOUR CODE HERE